In [ ]:
pip install numpy tensorflow pandas scikit-learn matplotlib

In [1]:
import os
import json
import cv2
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from keras.preprocessing.image import load_img, img_to_array
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
# Chargement du fichier CSV
df = pd.read_csv('/Users/quentinlandon/Desktop/GIT_Projet_theo/APP_IA_TL_QL_dev/informations_masques.csv')

# Extraction des chemins des images brutes et des masques
images_paths = df['pathimg'].values
masks_paths = df['pathmask'].values

# Fonction pour charger les images et les masques
def load_images_and_masks(images_paths, masks_paths):
    images = []
    masks = []
    for img_path, mask_path in zip(images_paths, masks_paths):
        img = load_img(img_path, target_size=(256, 256))
        mask = load_img(mask_path, target_size=(256, 256), color_mode='grayscale')
        images.append(img_to_array(img))
        masks.append(img_to_array(mask))
    return np.array(images), np.array(masks)

# Chargement des images et des masques
images, masks = load_images_and_masks(images_paths, masks_paths)

# Division des données en ensembles d'entraînement et de test (80% pour l'entraînement, 20% pour le test)
x_train, x_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Construction du modèle U-Net
def unet(input_shape):
    inputs = Input(input_shape)
    
    # Chemin contractant
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Fond
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Chemin expansif
    up4 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    up4 = concatenate([up4, conv2], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    up5 = concatenate([up5, conv1], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    # Couche de sortie
    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Création du modèle U-Net
input_shape = (256, 256, 3)  # Taille d'entrée
model = unet(input_shape)

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10


In [1]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt

# Charger une nouvelle image et son masque correspondant
new_image_path = "/Users/quentinlandon/Desktop/GIT_Projet_theo/APP_IA_TL_QL_dev/data/Img_test/Test1.png"
new_img = load_img(new_image_path, target_size=(256, 256))
new_img_array = img_to_array(new_img) / 255.0  # Normalisation entre 0 et 1
new_img_array = np.expand_dims(new_img_array, axis=0)

# Prédiction sur la nouvelle image
prediction = model.predict(new_img_array)[0]  # Première prédiction (on suppose une seule image en entrée)

# Afficher l'image originale
plt.subplot(1, 3, 1)
plt.imshow(new_img)
plt.title('Image originale')
plt.axis('off')

# Afficher la prédiction de segmentation
plt.subplot(1, 3, 3)
plt.imshow(prediction.squeeze(), cmap='gray')  # Utilisez 'gray' colormap pour les images en niveaux de gris
plt.title('Prédiction de segmentation')
plt.axis('off')

plt.show()

NameError: name 'model' is not defined